In [1]:
import pyrosetta
from pyrosetta import rosetta
from pyrosetta.rosetta import core
from pyrosetta.rosetta.core.import_pose import pose_from_file
from pyrosetta.rosetta.core.io.pdb import dump_pdb
from pyrosetta.rosetta.core.io.pdb import add_to_multimodel_pdb
from pyrosetta.rosetta.core.chemical import VariantType
from pyrosetta.io import poses_from_silent
from pyrosetta.rosetta.protocols.rna.denovo import RNA_DeNovoProtocol

import tempfile
import os
class Arguments:
    def __init__(self): pass

In [2]:
def mc_sample_single_process(args):
    input_pdbs = rosetta.utility.vector1_std_string()
    fasta_files = rosetta.utility.vector1_std_string()

    if os.path.exists(args.output_silent_file):
        os.remove(args.output_silent_file)
    if os.path.exists(args.output_pdb_file):
        os.remove(args.output_pdb_file)

    # Create a temporary file and write data to it
    with tempfile.NamedTemporaryFile(delete=True) as temp:
        temp.write(args.sequence.lower().encode())
        temp.flush()

        fasta_files.append(temp.name)

        rna_de_novo_setup = core.import_pose.RNA_DeNovoSetup()
        rna_de_novo_setup.set_fasta_files(fasta_files)
        rna_de_novo_setup.set_minimize_rna(True)

        rna_de_novo_setup.initialize_from_command_line()

        rna_de_novo_options = rna_de_novo_setup.options()
        rna_de_novo_options.set_nstruct(args.num_structs)
        rna_de_novo_options.set_silent_file(args.output_silent_file)

        directory = os.path.dirname(args.output_silent_file)
        os.makedirs(directory, exist_ok=True)
        directory = os.path.dirname(args.output_pdb_file)
        os.makedirs(directory, exist_ok=True)

        rna_de_novo_protocol = RNA_DeNovoProtocol(
            rna_de_novo_options, rna_de_novo_setup.rna_params()
        )

    rna_de_novo_pose = rna_de_novo_setup.pose()
    rna_de_novo_protocol.apply(rna_de_novo_pose)
    return rna_de_novo_pose

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


In [14]:
sample_modeller_parameters=Arguments()
sample_modeller_parameters.sequence="<seq>"
sample_modeller_parameters.output_silent_file="./rna_denovo.log"
sample_modeller_parameters.output_pdb_file="./rna_denovo_output.pdb"
sample_modeller_parameters.num_structs=8
sample_modeller_parameters.structs_per_job=4

In [16]:
print("sample_modeller_parameters:")
samplemodeldict=sample_modeller_parameters.__dict__
for k in samplemodeldict:
    print("\t",k+":",samplemodeldict[k])

sample_modeller_parameters:
	 sequence: <seq>
	 output_silent_file: ./rna_denovo.log
	 output_pdb_file: ./rna_denovo_output.pdb
	 num_structs: 8
	 structs_per_job: 4
